In [1]:
from utils.sqlite_interface import *
import numpy as np
import pandas as pd

In [2]:
DB_FILEPATH = "../data/sqlite/draft.db"

In [3]:
table_name = 'hs_pitchers_round_1_raw'
load_csv_to_table('../data/raw/round-1-hs-pitchers-drafted.csv', DB_FILEPATH, table_name)
print_tables(DB_FILEPATH)

CSV file '../data/raw/round-1-hs-pitchers-drafted.csv' located successfully.
Connected to SQLite database at '../data/sqlite/draft.db'.
'hs_pitchers_round_1_raw' table already exists. Data loading failed.
First 5 rows of 'hs_pitchers_round_1_raw':
(2025, '1', None, 6, 'FrRnd', 6, 'Pirates', 'Y', '$7250000', 'Seth Hernandez\xa0(minors)', 'P', None, None, None, None, None, None, None, None, None, None, None, None, 'HS', 'Corona HS (Corona CA)', '-9999')
(2025, '1', None, 25, 'FrRnd', 25, 'Padres', 'Y', '$3606600', 'Kruz Schoolcraft\xa0(minors)', 'P', None, None, None, None, None, None, None, None, None, None, None, None, 'HS', 'Sunset HS (Portland OR)', '-9999')
(2024, '1', None, 24, 'FrRnd', 24, 'Braves', 'Y', '$3553800', 'Cam Caminiti\xa0(minors)', 'P', None, None, None, None, None, None, None, None, None, None, None, None, 'HS', 'Saguaro HS (Scottsdale AZ)', '-9999')
(2024, '1', None, 25, 'FrRnd', 25, 'Padres', 'Y', '$3442100', 'Kash Mayfield\xa0(minors)', 'P', None, None, None, None,

In [4]:
query_db(f"PRAGMA table_info({table_name})", DB_FILEPATH)

Connected to SQLite database at '../data/sqlite/draft.db'.
Query returned 26 results.


[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Rnd', 'TEXT', 0, None, 0),
 (2, 'DT', 'REAL', 0, None, 0),
 (3, 'OvPck', 'INTEGER', 0, None, 0),
 (4, 'FrRnd', 'TEXT', 0, None, 0),
 (5, 'RdPck', 'INTEGER', 0, None, 0),
 (6, 'Tm', 'TEXT', 0, None, 0),
 (7, 'Signed', 'TEXT', 0, None, 0),
 (8, 'Bonus', 'TEXT', 0, None, 0),
 (9, 'Name', 'TEXT', 0, None, 0),
 (10, 'Pos', 'TEXT', 0, None, 0),
 (11, 'WAR', 'REAL', 0, None, 0),
 (12, 'G', 'REAL', 0, None, 0),
 (13, 'AB', 'REAL', 0, None, 0),
 (14, 'HR', 'REAL', 0, None, 0),
 (15, 'BA', 'REAL', 0, None, 0),
 (16, 'OPS', 'REAL', 0, None, 0),
 (17, 'G.1', 'REAL', 0, None, 0),
 (18, 'W', 'REAL', 0, None, 0),
 (19, 'L', 'REAL', 0, None, 0),
 (20, 'ERA', 'REAL', 0, None, 0),
 (21, 'WHIP', 'REAL', 0, None, 0),
 (22, 'SV', 'REAL', 0, None, 0),
 (23, 'Type', 'TEXT', 0, None, 0),
 (24, 'Drafted Out of', 'TEXT', 0, None, 0),
 (25, 'Name-additional', 'TEXT', 0, None, 0)]

In [5]:
signed_hs_pitchers_round_1 = query_db(f"SELECT year, name, ovpck, tm, bonus, war FROM {table_name} WHERE signed='Y'", DB_FILEPATH)
signed_hs_pitchers_round_1[0]

Connected to SQLite database at '../data/sqlite/draft.db'.
Query returned 418 results.


(2025, 'Seth Hernandez\xa0(minors)', 6, 'Pirates', '$7250000', None)

In [6]:
df = pd.DataFrame(signed_hs_pitchers_round_1)
df

,0,1,2,3,4,5
0,2025,Seth Hernandez (minors),6,Pirates,$7250000,NaN
1,2025,Kruz Schoolcraft (minors),25,Padres,$3606600,NaN
2,2024,Cam Caminiti (minors),24,Braves,$3553800,NaN
3,2024,Kash Mayfield (minors),25,Padres,$3442100,NaN
4,2024,Braylon Doughty (minors),36,Guardians,$2569200,NaN
...,...,...,...,...,...,...
413,1965,Les Rohr (minors),2,Mets,$50000,-0.3
414,1965,Joe Coleman (minors),3,Senators,$65000,23.2
415,1965,Rick James (minors),6,Cubs,$40000,-0.5
416,1965,Scott McDonald (minors),15,Orioles,$30000,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       418 non-null    int64  
 1   1       418 non-null    object 
 2   2       418 non-null    int64  
 3   3       418 non-null    object 
 4   4       399 non-null    object 
 5   5       252 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 19.7+ KB


In [8]:
df['year'] = df[0]
df['pick_number'] = df[2]
df['team'] = df[3]
df['player_name'] = df[1].apply(lambda x: x.removesuffix('(minors)').rstrip())
df['bonus'] = df[4].apply(lambda x: float(x.removeprefix('$')) if isinstance(x, str) else np.nan)
df['career_war'] = df[5]
df.drop(columns=[0,1,2,3,4,5], inplace=True)
df.dtypes

year             int64
pick_number      int64
team            object
player_name     object
bonus          float64
career_war     float64
dtype: object

In [9]:
df[df['bonus'].isna()]

,year,pick_number,team,player_name,bonus,career_war
228,1997,23,Expos,Donnie Bridges,NaN,NaN
231,1997,37,Expos,*Chris Stowe,NaN,NaN
232,1997,47,Expos,*T.J. Tucker,NaN,0.6
233,1997,48,Expos,*Shane Arthurs,NaN,NaN
241,1996,34,Red Sox,*Chris Reitsma,NaN,0.6
242,1996,35,Braves,*Jason Marquis,NaN,6.8
248,1995,30,Phillies,*Dave Coggin,NaN,0.2
260,1993,41,Blue Jays,*Jeremy Lee,NaN,NaN
268,1991,28,Cardinals,*Tom McKinnon,NaN,NaN
269,1991,30,Royals,*Jason Pruitt,NaN,NaN


In [ ]:
df['team'].value_counts()
# team-franchise mapping out of scope for now

team
Braves                   24
Padres                   19
Phillies                 19
Red Sox                  18
Tigers                   17
                         ..
White Sox via Yankees     1
Reds via Yankees          1
Cubs via Reds             1
Giants via Angels         1
Athletics via Rangers     1
Name: count, Length: 74, dtype: int64

In [11]:
df['player_name_clean'] = df['player_name'].apply(lambda x: x.removeprefix('*'))
df['bonus_int'] = df['bonus'].fillna(0).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               418 non-null    int64  
 1   pick_number        418 non-null    int64  
 2   team               418 non-null    object 
 3   player_name        418 non-null    object 
 4   bonus              399 non-null    float64
 5   career_war         252 non-null    float64
 6   player_name_clean  418 non-null    object 
 7   bonus_int          418 non-null    int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 26.3+ KB


In [12]:
df_cleaned = pd.DataFrame()
df_cleaned['year'] = df['year']
df_cleaned['pick_number'] = df['pick_number']
df_cleaned['team'] = df['team']
df_cleaned['player_name'] = df['player_name_clean']
df_cleaned['bonus'] = df['bonus_int']
df_cleaned['career_war'] = df['career_war']
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         418 non-null    int64  
 1   pick_number  418 non-null    int64  
 2   team         418 non-null    object 
 3   player_name  418 non-null    object 
 4   bonus        418 non-null    int64  
 5   career_war   252 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 19.7+ KB


In [13]:
# Import to SQL db